### Tratamento e Manipulação dos dados

As etapas deste notebook têm por objetivo garantir a integridade dos dados necessários para os cálculos da receita obtida em diferentes tipos de transações realizadas pelo PicPay, além de gerar arquivos em CSV com os resultados obtidos.

As tarefas realizadas foram divididas em:
1. Importação e inspeção dos dados
2. Cálculos da tabela "transactions"
3. Cálculos da tabela "transactions_installments"
4. Criação e exportação do arquivo final